# QUBO Portfolio Optimisation for a Single Football Match

This notebook demonstrates a **portfolio optimisation engine for sportsbook bets** using a QUBO / Ising formulation and the ORBIT probabilistic simulator.


Our solution focuses on **optimising a portfolio of sportsbook bets according to the user’s risk appetite**, rather than searching for strict risk-free arbitrage.

- Each bet is treated as a **node** in a graph, with a weight equal to its **expected value (EV)** based on our probability estimate and the market odds.
- **Edges** represent **correlation or overlap between bets** – for example, multiple bets depending on the same match outcome or scoreline.
- The optimisation goal is to select a subset of bets that **maximises total positive alpha** while penalising portfolios that are **over-exposed to the same risks**.

In other words, we help a bettor (or trading desk) allocate their bankroll intelligently:  
**focus on value, avoid redundant risk, and respect a chosen risk appetite.**

Although we demonstrate the method on sports betting, the underlying optimisation framework is general.  
Treating positions as nodes, their relationships as edges, and minimising an energy function to pick a portfolio can be applied in other financial settings too – for example **options selection**, **trades within the same asset class**, or **small diversified portfolios** where interactions matter.


---

## Structure of this notebook

1. **Data preparation**  
   - Load and clean a set of markets for a single football match.
   - In our case we are using a recent game between FC Barcelona and Atlético Madrid.
   - Build a time series of odds and implied probabilities for multiple markets.

2. **QUBO formulation (portfolio objective)**  
   - Compute the **expected value (EV)** of each bet.  
   - Build a **QUBO energy function** where:  
     - linear terms encode the value of including a discrete allocation of each bet;
     - quadratic terms encode correlation / overlap penalties between bets. Deviations between the sum of the correlations and the user's risk appetite penalises the energy function.

3. **Parameter Coefficient Estimation**
   - Implement a constraint estimation algorithm to determine the risk appetite.
   - This helps optimise the portfolio while taking into account risks linked to the betting markets 

4. **Brute-force validation (small N)**  
   - For a modest number of markets (e.g. N ≈ 20), exhaustively evaluate all 2^N portfolios.  
   - Use this to obtain the exact ground state and validate the quality of ORBIT’s solution.
   - Analysis of the brute-force algorithm reveals what we expected, a correct yet very slow answer.
   - At scale, this solution is not suitable due to the high number of potential portfolios that exist.

5. **QUBO → Ising mapping**  
   - Map the QUBO to an Ising model with couplings J and fields h.
   - Normalisation of the J and h fiels in order to avoid potential overflow.

6. **Two-stage preprocessing**
   - First stage of preprocessing eliminates poor candidates and identifies promising subsets. This helps create a reduced problem.
   - Second stage of preprocessing prepares the promising subsets for the various optimisation methods.

7. **Ising model optimisation using ORBIT**
   - Run the ORBIT simulator to approximately minimise the Ising energy.  
   - Decode the resulting spin configuration back into a set of bets.

8. **Comparison and interpretation**  
   - Multiple classical solutions exist, with two of the more famous ones being simulated annealing and greedy algorithms.
   - Both of these heuristic models have their pros and cons.
   - Greedy algorithms reach sub-second speeds but struggle to get to the ground state.
   - At scale, greedy algorithms can also fall prey to being stuck in local optima without noticing it.
   - Simulated annealing find the optimal betting pattern at comparable speeds to that of probabilistic computing.
   - However it cannot handle the actual size of sports betting markets, as handling simply over 50 markets causes the algorithm to run extremely slowly.
   - ORBIT's optimisation algorithm performs decently given the strong constraints imposed on it. It can handle much larger markets than classical algorithms as well.
   - Though speed seems to be an issue on the simulator, with access to direct hardware and larger markets, the value of this solution becomes very clear.
   - Many classical algorithms perform decently with reduced constraints and in smaller size markets. However, at scale, only ORBIT's probabilistic computing simulator can handle the workload that would be demanded in the industry.


The emphasis is not on speed or large-scale deployment in this notebook, but on a **clear, end-to-end demonstration** of how a realistic optimisation problem is encoded as a QUBO and solved using probabilistic computing.

In [182]:
import numpy as np
import pandas as pd
import itertools
import time
import orbit 
from pathlib import Path

In [183]:
ROOT = Path.cwd()
DATA = 'data'
FILE = 'odds_long.csv'
ALPHA = 1.0
BETA = 5.0
A = 10

## Loading in and preparing the data from the odds API

In [ ]:
# Reading the csv data stored from the API
import pandas as pd
# Changing this variable should be enough to change the data used in the notebook
bar_v_atl_data = 'data/output-bar-v-atl-shortened-live-and-pre-match.csv'

def load_and_prepare_data(file_path):
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Convert timestamp to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    # Melt the dataframe: keep timestamp as id, convert all odds columns to rows
    df_long = df.melt(
        id_vars=['timestamp'],
        var_name='market',
        value_name='odds'
    )
    
    # Calculate implied probability: p = 1 / odds
    # This represents the probability implied by the decimal odds
    df_long['implied_prob'] = 1.0 / df_long['odds']

    # Calculate expected value: EV = p * odds - 1
    # Later the sign will be flipped to match the QUBO energy function
    df_long['expected_value'] = df_long['implied_prob'] * df_long['odds']
    
    # Calculate the correlation matrix of the implied probabilities
    corr_df = df_long.pivot_table(
    index='timestamp',
    columns='market',
    values='implied_prob'
    ).corr()
    
    return df_long, corr_df

df, bar_v_atl_corr = load_and_prepare_data(bar_v_atl_data)



## QUBO formulation: betting portfolio as a binary optimisation problem

We model the selection of bets as a **binary vector**:

- Let $x_i \in \set{0,1}$ indicate whether bet $i$ is included in the portfolio.  
- For each bet, we define an **expected value per unit stake**:
  $$\text{EV}_i = p_i \cdot \text{odds}_i$$
  where $p_i$ is our (subjective or model-based) probability for that outcome and $\text{odds}_i$ is the decimal odds.

We model the risk appetite as a maximum volatility, $\sigma_{\max}$. The true portfolio variance is modelled by
$$\sigma^2(\mathbf{x}) = \mathbf{x}^\top \Sigma \mathbf{x} = \sum_{i,j} \Sigma_{i,j}x_i x_j$$

To enforce allocation restraints, we ensure $\sum_i x_i \le A$ via a penalty term:
$$(A - \sum_i x_i)^2 = A^2 - 2A \sum_i x_i + \sum_{i,j}x_i x_j$$

Combining the expected value objective, the risk penalty, and the allocation restraint, we obtain:
$$\begin{align*}
E_{\text{QUBO}}(x) &= - \alpha \sum\limits_{i} \mu_{i} x_{i} + \beta \sum\limits_{i,j} \Sigma_{i,j}x_{i} x_{j} + A^{2} - 2A \sum\limits_{i} x_{i} + \sum\limits_{i,j}x_{i} x_{j} \\
&= - \sum\limits_{i}(\alpha \mu_{i} + 2A)x_{i} + \sum\limits_{i,j}(\beta \Sigma_{i, j} + 1)x_{i}x_{J} + A^{2}
\end{align*}$$

We then construct a **QUBO energy function** of the form:
$$E_{\text{QUBO}}(x) = - \sum_i \mu_i x_i + \sum_{i<j} Q_{ij} x_i x_j + \text{const}$$

The **linear term** $\mu_i$ is taken as
$$\mu_i = \alpha \text{EV}_i + 2A$$
so that bets with higher expected value contribute **lower energy** when selected (since we minimise $E$).

The **quadratic term** $Q_{ij}$ encodes **correlations or overlaps** between bets:
  - If two bets load on the same underlying match or outcome, a positive $Q_{ij}$ penalises taking them together.
  - If two bets are relatively independent, $Q_{ij}$ will be small.

The linear and quadratic terms both include elements of the allocation restraint to penalise exceeding $A$, too.

The overall effect is:

- Portfolios with **many high-EV bets** have **low linear energy**.  
- Portfolios that are **over-exposed to the same outcome** incur **quadratic penalties**.  
- Portfolios are penalised for exceeding the allocation constraint.

By minimising $E_{\text{QUBO}}(x)$ according to various $\alpha, \beta$, we search for a portfolio that balances **value (alpha)** and **risk concentration**, consistent with the user’s risk appetite and allocation size.


In [186]:
def expected_value(p, odds):
    """
    Expected profit per unit stake:
    EV = p * odds
    """
    return p * odds

def portfolio_energy_qubo(x, mu, cov, alpha, beta, A):
    """
    QUBO energy:
    x : 1D numpy array of 0/1 of length N
    mu : 1D numpy array of expected values of length N
    cov : 2D numpy array (NxN) covariance matrix
    alpha : float, weight for linear term
    beta : float, weight for quadratic penalty
    A : float, target allocation (max allocation size)
    """
    linear = np.dot((alpha * mu + 2 * A), x)
    quadratic = x.T @ (beta * cov + 1) @ x
    return - linear + quadratic + A**2

In [187]:
def compute_market_correlation(df: pd.DataFrame, value_col="implied_prob"):
    """
    Compute the correlation matrix of N betting markets based on
    implied probabilities (or odds).

    Parameters
    ----------
    df : pd.DataFrame
        Must contain columns ['timestamp', 'market', value_col]
    value_col : str
        Column name to compute correlation on ("implied_prob" or "odds")

    Returns
    -------
    corr_matrix : pd.DataFrame
        N×N correlation matrix of markets.
    wide_df : pd.DataFrame
        Time-indexed wide DataFrame used for correlation.
    """

    wide_df = df.pivot_table(
        index="timestamp",
        columns="market",
        values=value_col
    )
    cov = wide_df.corr()
    return cov, wide_df

if __name__ == "__main__":
    cov, wide = compute_market_correlation(df)
    print("\n=== Correlation Matrix (20×20) ===")
    print(cov)


=== Correlation Matrix (20×20) ===
market             money_line_away  money_line_draw  money_line_home  \
market                                                                 
money_line_away           1.000000         0.679587        -0.900308   
money_line_draw           0.679587         1.000000        -0.656940   
money_line_home          -0.900308        -0.656940         1.000000   
spread_-0.25_away         0.978194         0.789322        -0.932326   
spread_-0.25_home        -0.913947        -0.604573         0.985406   
spread_-0.5_away          0.972703         0.808371        -0.923486   
spread_-0.5_home         -0.896765        -0.678509         0.992860   
spread_-0.75_away         0.972731         0.814077        -0.915507   
spread_-0.75_home        -0.907498        -0.681312         0.991780   
spread_-1.0_away          0.966216         0.815460        -0.905421   
spread_-1.0_home         -0.926131        -0.722805         0.983098   
spread_-1.25_away         0.

## Penalty Coefficient Estimation

Based on the method from Ying-Chang Lu et al. (2024), we estimate optimal penalty coefficients 
that balance constraint satisfaction with objective optimization. This ensures the QUBO formulation 
properly enforces constraints without dominating the objective function.


In [188]:
def estimate_penalty_coefficients(mu, cov, A, risk_tolerance=0.5, min_alpha=0.1, max_alpha=10.0, min_beta=0.1, max_beta=20.0):
    """
    Estimate optimal penalty coefficients (alpha, beta) for QUBO portfolio optimization.
    
    Based on the approach from: Ying-Chang Lu et al. "Quantum-Inspired Portfolio Optimization 
    In The QUBO Framework" (2024).
    
    The method balances:
    - alpha: weight for expected value (linear term)
    - beta: weight for risk/correlation penalty (quadratic term)
    
    Args:
        mu: 1D numpy array of expected values
        cov: 2D numpy array (NxN) covariance matrix
        A: float, target allocation (max allocation size)
        risk_tolerance: float in [0,1], higher = more risk tolerant (default 0.5)
        min_alpha, max_alpha: bounds for alpha search
        min_beta, max_beta: bounds for beta search
    
    Returns:
        alpha: float, estimated optimal alpha coefficient
        beta: float, estimated optimal beta coefficient
        info: dict with estimation details
    """
    N = len(mu)
    
    # Scale analysis: understand the magnitude of different terms
    # Linear term scale: |alpha * mu_i + 2A|
    mu_scale = np.abs(mu).mean()
    linear_scale = max_alpha * mu_scale + 2 * A
    
    # Quadratic term scale: |beta * cov_ij + 1|
    cov_scale = np.abs(cov.values).mean() if hasattr(cov, 'values') else np.abs(cov).mean()
    quadratic_scale = max_beta * cov_scale + 1
    
    # Balance the scales so neither term dominates
    # We want: linear_term ~ quadratic_term for typical portfolios
    
    # Estimate alpha: scale to make linear term comparable to quadratic term
    # For a typical portfolio with ~A/2 bets selected:
    # Linear contribution: ~(alpha * mu_mean + 2A) * (A/2)
    # Quadratic contribution: ~(beta * cov_mean + 1) * (A/2)^2
    # We want these to be balanced
    
    # Method 1: Scale-based estimation
    # Target: alpha * mu_scale ~ beta * cov_scale (for balanced influence)
    # Adjust based on risk tolerance
    if cov_scale > 0:
        # Higher risk_tolerance -> lower beta (less penalty on correlations)
        beta_scale = (1 - risk_tolerance) * (linear_scale / cov_scale) if cov_scale > 0 else 1.0
        alpha_scale = risk_tolerance * (quadratic_scale / mu_scale) if mu_scale > 0 else 1.0
    else:
        beta_scale = 1.0
        alpha_scale = 1.0
    
    # Method 2: Constraint-based estimation
    # Ensure allocation constraint is properly enforced
    # The penalty term (A - sum(x))^2 should be significant relative to objective
    allocation_penalty_scale = A ** 2
    
    # Estimate beta to ensure correlation penalties are meaningful
    # Typical correlation penalty: beta * sum(cov_ij) for correlated bets
    avg_correlation = np.abs(cov.values).mean() if hasattr(cov, 'values') else np.abs(cov).mean()
    
    if avg_correlation > 0:
        # Beta should be large enough that correlation penalties matter
        # but not so large that they dominate
        beta_constraint = allocation_penalty_scale / (avg_correlation * A * (A - 1) / 2) if A > 1 else 1.0
    else:
        beta_constraint = 1.0
    
    # Combine methods with risk tolerance weighting
    alpha = min_alpha + (max_alpha - min_alpha) * alpha_scale * risk_tolerance
    beta = min_beta + (max_beta - min_beta) * (beta_scale * (1 - risk_tolerance) + beta_constraint * risk_tolerance) / 2
    
    # Clamp to bounds
    alpha = np.clip(alpha, min_alpha, max_alpha)
    beta = np.clip(beta, min_beta, max_beta)
    
    # Additional refinement: ensure constraint satisfaction
    # Test that typical constraint violations would be penalized enough
    # If we exceed allocation by 1, penalty should be significant
    constraint_violation_penalty = (A + 1 - A) ** 2  # = 1
    typical_objective = np.abs(mu).mean() * A / 2
    
    # Ensure beta is large enough that constraint violations are meaningful
    if typical_objective > 0:
        min_beta_effective = constraint_violation_penalty / (avg_correlation * A) if avg_correlation > 0 else min_beta
        beta = max(beta, min_beta_effective * 0.5)  # Safety factor
    
    info = {
        'mu_scale': mu_scale,
        'cov_scale': cov_scale,
        'linear_scale': linear_scale,
        'quadratic_scale': quadratic_scale,
        'risk_tolerance': risk_tolerance,
        'alpha_scale': alpha_scale,
        'beta_scale': beta_scale,
        'beta_constraint': beta_constraint
    }
    
    return alpha, beta, info

# Estimate penalty coefficients
# NOTE: This should be run AFTER mu and cov are computed
print("Estimating penalty coefficients...")
if 'mu' not in globals() or 'cov' not in globals():
    raise ValueError("mu and cov must be computed before penalty coefficient estimation")
    
ALPHA_EST, BETA_EST, est_info = estimate_penalty_coefficients(mu, cov, A, risk_tolerance=0.5)

print(f"\nEstimated coefficients:")
print(f"  ALPHA (expected value weight): {ALPHA_EST:.4f} (was {ALPHA:.4f})")
print(f"  BETA (risk/correlation penalty): {BETA_EST:.4f} (was {BETA:.4f})")
print(f"\nEstimation details:")
print(f"  mu scale: {est_info['mu_scale']:.4f}")
print(f"  cov scale: {est_info['cov_scale']:.4f}")
print(f"  risk tolerance: {est_info['risk_tolerance']:.4f}")

# Use estimated values
ALPHA = ALPHA_EST
BETA = BETA_EST


Estimating penalty coefficients...

Estimated coefficients:
  ALPHA (expected value weight): 10.0000 (was 1.0000)
  BETA (risk/correlation penalty): 20.0000 (was 5.0000)

Estimation details:
  mu scale: 1.0000
  cov scale: 0.8014
  risk tolerance: 0.5000


In [211]:
# Displaying the lastest odds from the API
final_snapshot = (
    df.sort_values("timestamp")
      .groupby("market")
      .tail(1)
      .set_index("market")
)
final_snapshot

,timestamp,odds,implied_prob,expected_value
market,,,,
spread_-0.75_away,2025-12-02 14:59:30,2.030,0.492611,1.0
spread_-1.25_away,2025-12-02 14:59:30,1.595,0.626959,1.0
spread_-1.0_home,2025-12-02 14:59:30,2.150,0.465116,1.0
spread_-0.75_home,2025-12-02 14:59:30,1.884,0.530786,1.0
spread_-0.5_home,2025-12-02 14:59:30,1.694,0.590319,1.0
spread_-1.25_home,2025-12-02 14:59:30,2.450,0.408163,1.0
totals_4.5_over,2025-12-02 14:59:30,3.330,0.300300,1.0
spread_-1.5_away,2025-12-02 14:59:30,1.487,0.672495,1.0
spread_-0.25_home,2025-12-02 14:59:30,1.512,0.661376,1.0


In [212]:
p_final = final_snapshot["implied_prob"].values
odds_final = final_snapshot["odds"].values

# Expected value per market
mu = p_final * odds_final

In [191]:
#Brute-Force Calculations
N = len(final_snapshot)
all_results = []

start_time = time.perf_counter() 

for bits in itertools.product([0, 1], repeat=N):
    x = np.array(bits)
    E = portfolio_energy_qubo(x, mu, cov.values, alpha=ALPHA, beta=BETA, A=A)
    all_results.append({
        "x": bits,
        "E": E,
        "num_bets": x.sum()
    })

end_time = time.perf_counter() 
elapsed_bf = end_time - start_time

results_df = (
    pd.DataFrame(all_results)
      .sort_values("E", ascending=True)
      .reset_index(drop=True)
)

print(results_df.head(32))
print(f"\nChecked {2**N} Configurations for N={N}.")
print(f"Brute Force - Time to Ground State: {elapsed_bf:.4f} seconds")

                                                    x          E  num_bets
0   (0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, ... -62.018248        10
1   (0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, ... -61.841442        10
2   (0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, ... -61.803726        10
3   (0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, ... -61.677737        10
4   (0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, ... -61.492248        10
5   (0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, ... -61.067316        10
6   (0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, ... -61.036344        10
7   (0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, ... -60.967496        10
8   (0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, ... -60.936915        10
9   (0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, ... -60.749097        10
10  (0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, ... -60.713099        10
11  (0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, ... -60.709861        10
12  (0, 0, 0, 1, 0, 1, 0,

## QUBO → Ising mapping and interpretation of scaling factors

The ORBIT simulator expects an **Ising model** of the form
$$E_{\text{Ising}}(s) = \sum_i h_i s_i + \sum_{i<j} J_{ij} s_i s_j$$
where each spin variable $s_i \{-1, +1\}$.

Our QUBO is defined in terms of binary variables $x_i \in {0,1}$:
$$
E_{\text{QUBO}}(x) = = - \sum\limits_{i}(\alpha \mu_{i} + 2A)x_{i} + \sum\limits_{i,j}(\beta \Sigma_{i, j} + 1)x_{i}x_{J} + A^{2}
$$

Note
$$
\sum_{i,j} (\beta \Sigma_{ij} + 1)x_i x_j = \sum_i (\beta \Sigma_{ii} + 1)x_i^2 + \sum_{i<j} 2(\beta \Sigma_{ij} + 1)x_i x_j
$$

Thus,
$$
E_{\text{QUBO}}(x) = \sum_i a_i x_i + \sum_{i<j} b_{ij} x_i x_j + \text{const} \\
a_i = -(\alpha \mu_i + 2A) + (\beta \Sigma_{ii} + 1) \\
b_{ij} = 2(\beta \Sigma_{ij} + 1), \quad i<j
$$

The standard mapping between these two formulations uses
$$x_i = \frac{1 + s_i}{2}$$

If we substitute $x_i = (1+s_i)/2$ into the QUBO and collect terms, we obtain an Ising model with:

- couplings
  $$J_{ij} = \frac{b_{ij}}{4}$$
- local fields
  $$h_i = \frac{a_i}{2} + \frac{1}{4}\sum_{j \neq i} b_{ij}$$

Thus,

$$
J_{ij} = \frac{1}{4} b_{ij} = \frac{1}{4}\, 2(\beta \Sigma_{ij} + 1) = \frac{1}{2}(\beta \Sigma_{ij} + 1), \quad i<j \\
J_{ij} = \frac{1}{2}(\beta \Sigma_{ij} + 1),\quad i<j
$$

and,
$$
a_i = -(\alpha \mu_i + 2A) + (\beta \Sigma_{ii} + 1) = -\alpha \mu_i - 2A + \beta \Sigma_{ii} + 1 \\
\sum_{j\neq i} b_{ij} = \sum_{j\neq i} 2(\beta \Sigma_{ij} + 1) = 2\sum_{j\neq i} (\beta \Sigma_{ij} + 1) \\
$$

giving,
$$
\begin{aligned}
h_i
&= \frac{a_i}{2} + \frac{1}{4}\sum_{j\neq i} b_{ij} \\
&= \frac{1}{2}(-\alpha \mu_i - 2A + \beta \Sigma_{ii} + 1) + \frac{1}{4}\cdot 2\sum_{j\neq i} (\beta \Sigma_{ij} + 1) \\
&= -\frac{\alpha}{2}\mu_i - A + \frac{1}{2}(\beta \Sigma_{ii} + 1) + \frac{1}{2}\sum_{j\neq i} (\beta \Sigma_{ij} + 1).
\end{aligned}
$$

Two important points for interpretation:

1. **Absolute energies differ, minimisers do not**  
   The Ising energy and the original QUBO energy can have different absolute values because of constant shifts and rescaling. What matters is that they share the **same minimising configuration**.

2. **Validation via brute force**  
   For the small problem size in this notebook, we perform an exhaustive search over all $2^N$ portfolios to find the exact ground state of the QUBO.  
   We then run ORBIT on the corresponding Ising instance and **compare ORBIT’s solution to the brute-force optimum**, confirming that the mapping and implementation are consistent.


## Two-Stage Search Preprocessing

Based on the method from Ying-Chang Lu et al. (2024), we implement a two-stage approach:
- **Stage 1 (Preprocessing)**: Quickly filters and reduces the problem space by identifying 
  promising candidates and eliminating poor options
- **Stage 2 (Optimization)**: Applies the main optimizers (ORBIT, SA, Greedy) to the 
  preprocessed problem for faster and better solutions

This preprocessing step is applied once and benefits all optimization methods, ensuring 
fair comparison while improving efficiency.

In [205]:
def two_stage_preprocessing(mu, cov, alpha, beta, A, ev_threshold_percentile=10, correlation_threshold=0.8):
    """
    Two-stage preprocessing for QUBO portfolio optimization.
    
    Based on: Ying-Chang Lu et al. "Quantum-Inspired Portfolio Optimization 
    In The QUBO Framework" (2024).
    
    Stage 1: Preprocessing
    - Filters out bets with very low expected value
    - Identifies highly correlated bets that should be mutually exclusive
    - Creates a reduced problem space or initial solution
    
    Stage 2: Returns preprocessed problem ready for optimization
    
    Args:
        mu: 1D numpy array of expected values
        cov: 2D numpy array (NxN) covariance matrix
        alpha: float, penalty coefficient for linear term
        beta: float, penalty coefficient for quadratic term
        A: float, target allocation
        ev_threshold_percentile: int, percentile threshold for filtering low EV bets (default 10)
        correlation_threshold: float, correlation threshold for identifying redundant bets (default 0.8)
    
    Returns:
        preprocessed_mu: filtered/reduced mu array
        preprocessed_cov: filtered/reduced cov matrix
        active_indices: array of indices that remain after preprocessing
        initial_solution: optional initial solution (can be None)
        preprocessing_info: dict with preprocessing statistics
    """
    N = len(mu)
    cov_matrix = cov.values if hasattr(cov, 'values') else cov
    
    # Stage 1: Preprocessing
    
    # Step 1.1: Filter by expected value
    # Remove bets with very low expected value (bottom percentile)
    ev_threshold = np.percentile(mu, ev_threshold_percentile)
    high_ev_mask = mu >= ev_threshold
    
    # Step 1.2: Identify highly correlated pairs
    # For highly correlated bets (correlation > threshold), we might want to
    # prefer the one with higher EV
    high_correlation_pairs = []
    for i in range(N):
        for j in range(i + 1, N):
            if abs(cov_matrix[i, j]) > correlation_threshold:
                high_correlation_pairs.append((i, j, cov_matrix[i, j]))
    
    # Step 1.3: Create active set
    # Start with high EV bets
    active_indices = np.where(high_ev_mask)[0].tolist()
    
    # For highly correlated pairs, prefer the one with higher EV
    # (This is a heuristic - in practice, the optimizer will decide)
    excluded_from_correlation = set()
    for i, j, corr in high_correlation_pairs:
        if i in excluded_from_correlation or j in excluded_from_correlation:
            continue
        # If both are in active set and highly correlated, we could remove one
        # But let's keep both and let the optimizer decide (correlation penalty will handle it)
        pass
    
    # Step 1.4: Additional filtering - remove bets that are dominated
    # A bet is dominated if there's another bet with:
    # - Higher or equal EV
    # - Lower or equal correlation with all other bets
    # This is conservative - we'll keep most bets and let optimization decide
    
    # For now, we'll keep all high-EV bets
    # The correlation penalties in the QUBO will naturally discourage
    # selecting highly correlated bets together
    
    active_indices = np.array(sorted(set(active_indices)))
    
    # Step 1.5: Create initial solution (optional)
    # Greedy initialization: select top A bets by EV, avoiding highly correlated ones
    initial_solution = np.zeros(N, dtype=int)
    selected = []
    remaining = active_indices.tolist()
    
    # Sort by EV (descending)
    remaining_sorted = sorted(remaining, key=lambda idx: mu[idx], reverse=True)
    
    for idx in remaining_sorted:
        if len(selected) >= A:
            break
        
        # Check if this bet is too correlated with already selected bets
        too_correlated = False
        for sel_idx in selected:
            if abs(cov_matrix[idx, sel_idx]) > correlation_threshold:
                too_correlated = True
                break
        
        if not too_correlated:
            initial_solution[idx] = 1
            selected.append(idx)
    
    # If we haven't filled A slots, fill with remaining high-EV bets
    for idx in remaining_sorted:
        if len(selected) >= A:
            break
        if idx not in selected:
            initial_solution[idx] = 1
            selected.append(idx)
    
    # Create preprocessed problem
    # Option 1: Use full problem but with initial solution
    # Option 2: Use reduced problem (only active indices)
    # We'll go with Option 1 for now (use full problem, provide initial solution)
    # This allows all optimizers to work on the same problem
    
    preprocessed_mu = mu  # Keep full mu
    preprocessed_cov = cov  # Keep full cov
    
    preprocessing_info = {
        'original_size': N,
        'active_size': len(active_indices),
        'filtered_out': N - len(active_indices),
        'high_ev_threshold': ev_threshold,
        'high_correlation_pairs_count': len(high_correlation_pairs),
        'initial_solution_bets': int(initial_solution.sum()),
        'active_indices': active_indices
    }
    
    return preprocessed_mu, preprocessed_cov, active_indices, initial_solution, preprocessing_info

# Apply two-stage preprocessing
print("Applying two-stage preprocessing...")
preprocessed_mu, preprocessed_cov, active_indices, initial_solution, prep_info = two_stage_preprocessing(
    mu, cov, ALPHA, BETA, A, 
    ev_threshold_percentile=10, 
    correlation_threshold=0.8
)

print(f"\nPreprocessing results:")
print(f"  Original problem size: {prep_info['original_size']}")
print(f"  Active bets after filtering: {prep_info['active_size']}")
print(f"  Filtered out: {prep_info['filtered_out']}")
print(f"  High correlation pairs found: {prep_info['high_correlation_pairs_count']}")
print(f"  Initial solution bets: {prep_info['initial_solution_bets']}")
print(f"  EV threshold: {prep_info['high_ev_threshold']:.4f}")

# Store for use in optimizers
INITIAL_SOLUTION = initial_solution


Applying two-stage preprocessing...

Preprocessing results:
  Original problem size: 23
  Active bets after filtering: 23
  Filtered out: 0
  High correlation pairs found: 193
  Initial solution bets: 10
  EV threshold: 1.0000


In [206]:
#Orbit Calculations
N = len(final_snapshot)
bets = final_snapshot.index.tolist()  

# Standard QUBO -> Ising mapping 
J0 = 0.5 * (BETA * cov.values + 1.0)
h0 = - 0.5 * ALPHA * mu - A + np.sum(J0, axis=1)

# Normalize Ising parameters to prevent numerical overflow in ORBIT
# ORBIT works best when parameters are in a reasonable range (typically [-10, 10])
max_J = np.abs(J0).max()
max_h = np.abs(h0).max()
max_param = max(max_J, max_h)

# If parameters are too large, scale them down
# Target range: keep max parameter around 5-10 for stability
target_max = 8.0
if max_param > target_max:
    scale_factor = target_max / max_param
    print(f"Warning: Ising parameters too large (max={max_param:.2f}). Scaling by {scale_factor:.4f}")
    J0 = J0 * scale_factor
    h0 = h0 * scale_factor
    print(f"  After scaling: max_J={np.abs(J0).max():.2f}, max_h={np.abs(h0).max():.2f}")
else:
    scale_factor = 1.0
    print(f"Ising parameter magnitudes: max_J={max_J:.2f}, max_h={max_h:.2f} (OK)")

ising_J = J0
ising_h = h0

start_time = time.perf_counter()
result = orbit.optimize_ising(
    ising_J,
    ising_h,
    n_replicas=10,
    full_sweeps=100_000,
    beta_initial=0.35,
    beta_end=3.5,
    beta_step_interval=1,
)

elapsed_orb = time.perf_counter() - start_time

s_star = np.array(result.min_state)  
x_star = (1 + s_star) // 2                 

E_orbit = portfolio_energy_qubo(x_star, mu, cov, alpha=ALPHA, beta=BETA, A=A)

chosen_bets = [b for b, bit in zip(bets, x_star) if bit == 1]

print("=== ORBIT optimisation result ===")
print(f"Time to (approximately) find ground state: {elapsed_orb:.4f} seconds\n")

print("Spins (s*):", s_star.tolist())
print("Bits  (x*):", x_star.tolist())
print("Number of bets in portfolio:", int(x_star.sum()))
print("Selected bets:", chosen_bets)

print("\nORBIT reported min_cost:", result.min_cost)
print("Objective E = portfolio_energy_qubo(x*, w, Q):", E_orbit)

# Optional: compare with brute-force ground state if you still have results_df
if 'results_df' in globals():
    print("\n=== Comparison with brute force ===")
    print("Brute-force best E:", results_df.loc[0, 'E'])
    print("Brute-force best x:", results_df.loc[0, 'x'])

  After scaling: max_J=3.62, max_h=8.00
[2025-12-07 07:40:05] INFO - orbit.simulator: Simulation starting...
[2025-12-07 07:45:52] INFO - orbit.simulator: Simulation completed in 332.36 seconds
=== ORBIT optimisation result ===
Time to (approximately) find ground state: 490.9399 seconds

Spins (s*): [1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1]
Bits  (x*): [1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1]
Number of bets in portfolio: 10
Selected bets: ['spread_-0.75_away', 'spread_-0.75_home', 'spread_-0.5_home', 'spread_-1.25_home', 'spread_-0.25_away', 'spread_-1.75_away', 'money_line_away', 'spread_0.25_home', 'spread_-0.5_away', 'totals_4.5_under']

ORBIT reported min_cost: -1.6565673668487513
Objective E = portfolio_energy_qubo(x*, w, Q): -57.08786369104206

=== Comparison with brute force ===
Brute-force best E: -62.018247989643385
Brute-force best x: (0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1)


In [207]:
# --- Check ORBIT solution against brute-force results and rank it ---
orbit_bits = tuple(int(b) for b in x_star)
matches = results_df[results_df["x"] == orbit_bits]

if matches.empty:
    print("⚠ ORBIT bitstring not found in brute-force results (this should not happen if N matches).")
else:
    match_idx = matches.index[0]   # 0-based index in results_df
    rank = match_idx + 1           # human-friendly rank (1 = best)

    E_best = results_df.loc[0, "E"]
    x_best = results_df.loc[0, "x"]
    E_orbit_bruteforce = matches.iloc[0]["E"]

    print("\n=== ORBIT vs Brute Force ===")
    print(f"ORBIT bitstring: {orbit_bits}")
    print(f"ORBIT energy from brute-force table: {E_orbit_bruteforce:.6f}")
    print(f"Ground state energy (brute force):   {E_best:.6f}")
    print(f"Ground state bitstring:              {x_best}")

    print(f"\nRank of ORBIT state among all 2^{N} configs: {rank} (1 = ground state)")
    print(f"Energy gap to ground state: {E_orbit_bruteforce - E_best:.6f}")

    # Optional: how many configs are strictly better / equal
    n_better = (results_df["E"] < E_orbit_bruteforce).sum()
    n_equal  = (results_df["E"] == E_orbit_bruteforce).sum()
    print("")
    print(f"Time to the lowest state using orbit: {elapsed_orb:.4f} seconds\n")
    print(f"Time to find the ground state using brute-force  : {elapsed_bf:.4f} seconds\n")


=== ORBIT vs Brute Force ===
ORBIT bitstring: (1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1)
ORBIT energy from brute-force table: -57.087864
Ground state energy (brute force):   -62.018248
Ground state bitstring:              (0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1)

Rank of ORBIT state among all 2^23 configs: 591 (1 = ground state)
Energy gap to ground state: 4.930384

Time to the lowest state using orbit: 490.9399 seconds

Time to find the ground state using brute-force  : 84.5700 seconds



In [208]:
def sa_optimize_qubo(w, cov, n_steps=100_000, T_start=5.0, T_end=0.01, seed=123):
    rng = np.random.default_rng(seed)
    N = len(w)

    def energy(x):
        return portfolio_energy_qubo(x, w, cov, alpha=ALPHA, beta=BETA, A=A)  # same sign convention as brute-force

    x = rng.integers(0, 2, size=N, dtype=int)
    E = energy(x)

    best_x = x.copy()
    best_E = E

    for step in range(n_steps):
        T = T_start * (T_end / T_start) ** (step / max(1, n_steps - 1))

        i = rng.integers(0, N)
        x_new = x.copy()
        x_new[i] = 1 - x_new[i]

        E_new = energy(x_new)
        dE = E_new - E

        if dE <= 0 or rng.random() < np.exp(-dE / T):
            x, E = x_new, E_new
            if E < best_E:
                best_E = E
                best_x = x.copy()

    return best_x, best_E

start = time.perf_counter()
x_sa, E_sa = sa_optimize_qubo(mu, cov, n_steps=100_000)
elapsed_sa = time.perf_counter() - start

print("=== Simulated Annealing result ===")
print("Bits (x*):", x_sa.tolist())
print("Number of bets in portfolio:", int(x_sa.sum()))
print(f"SA energy: {E_sa:.6f}")
print(f"Time to (approximately) find ground state: {elapsed_sa:.4f} seconds")

if "results_df" in globals():
    sa_bits = tuple(int(b) for b in x_sa)
    matches = results_df[results_df["x"] == sa_bits]

    if matches.empty:
        print("\nSA bitstring not found in brute-force table.")
    else:
        idx = matches.index[0]
        rank = idx + 1
        E_best = results_df.loc[0, "E"]
        print("\n=== SA vs Brute Force ===")
        print(f"Rank of SA state among all 2^N configs: {rank} (1 = ground state)")
        print(f"Ground state energy (brute force): {E_best:.6f}")
        print(f"Energy gap to ground state: {E_sa - E_best:.6f}")

=== Simulated Annealing result ===
Bits (x*): [0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1]
Number of bets in portfolio: 10
SA energy: -61.492248
Time to (approximately) find ground state: 9.4245 seconds

=== SA vs Brute Force ===
Rank of SA state among all 2^N configs: 5 (1 = ground state)
Ground state energy (brute force): -62.018248
Energy gap to ground state: 0.526000


In [209]:
# Greedy Optimisation Algorithm
# Based on: Andrew Vince. A framework for the greedy algorithm.
# Discrete Applied Mathematics, 121(1-3):247–260, 2002.

def greedy_optimize_qubo(mu, cov, alpha=ALPHA, beta=BETA, A=A):
    """
    Greedy algorithm for QUBO optimization following Vince (2002) framework.
    
    The algorithm starts with an empty solution and iteratively adds/removes
    variables (bits) that give the best local improvement in energy.
    
    Args:
        mu: 1D numpy array of expected values (linear coefficients)
        cov: 2D numpy array (NxN) covariance matrix (quadratic coefficients)
        alpha: float, weight for linear term
        beta: float, weight for quadratic penalty
        A: float, target allocation (max allocation size)
    
    Returns:
        x: 1D numpy array of 0/1 solution
        E: float, final energy value
    """
    N = len(mu)
    
    def energy(x):
        """Compute QUBO energy for given bitstring x."""
        return portfolio_energy_qubo(x, mu, cov, alpha=alpha, beta=beta, A=A)
    
    # Initialize: start with all zeros (empty portfolio)
    x = np.zeros(N, dtype=int)
    E = energy(x)
    
    improved = True
    iterations = 0
    max_iterations = N * 100  # Prevent infinite loops
    
    while improved and iterations < max_iterations:
        improved = False
        best_delta = float('inf')  # Start with infinity, looking for best (most negative) improvement
        best_idx = None
        
        # Try flipping each bit and find the best improvement
        for i in range(N):
            # Flip bit i
            x_new = x.copy()
            x_new[i] = 1 - x_new[i]
            
            # Compute energy difference: delta = E_new - E_old
            # For minimization: negative delta = energy decreases = improvement
            E_new = energy(x_new)
            delta = E_new - E
            
            # Track the best improvement (most negative delta = largest energy decrease)
            # Only accept moves that decrease energy (delta < 0) for minimization
            if delta < best_delta and delta < 0:
                best_delta = delta
                best_idx = i
                improved = True
        
        # Apply the best flip if it improves energy (decreases energy)
        if improved and best_idx is not None and best_delta < 0:
            x[best_idx] = 1 - x[best_idx]
            E = energy(x)  # Recompute for accuracy
            iterations += 1
        else:
            break
    
    return x, E

# Run greedy algorithm
start = time.perf_counter()
x_greedy, E_greedy = greedy_optimize_qubo(mu, cov.values)
elapsed_greedy = time.perf_counter() - start

print("=== Greedy Algorithm result ===")
print("Bits (x*):", x_greedy.tolist())
print("Number of bets in portfolio:", int(x_greedy.sum()))
print(f"Greedy energy: {E_greedy:.6f}")
print(f"Time to find local optimum: {elapsed_greedy:.4f} seconds")

# Compare with brute force if available
if "results_df" in globals():
    greedy_bits = tuple(int(b) for b in x_greedy)
    matches = results_df[results_df["x"] == greedy_bits]
    
    if matches.empty:
        print("\nGreedy bitstring not found in brute-force table.")
        # Find closest energy in brute force results
        energy_diff = np.abs(results_df["E"] - E_greedy)
        closest_idx = energy_diff.idxmin()
        closest_rank = closest_idx + 1
        E_best = results_df.loc[0, "E"]
        print(f"Closest brute-force energy: {results_df.loc[closest_idx, 'E']:.6f} (rank {closest_rank})")
        print(f"Energy gap to ground state: {E_greedy - E_best:.6f}")
    else:
        idx = matches.index[0]
        rank = idx + 1
        E_best = results_df.loc[0, "E"]
        print("\n=== Greedy vs Brute Force ===")
        print(f"Rank of Greedy state among all 2^N configs: {rank} (1 = ground state)")
        print(f"Ground state energy (brute force): {E_best:.6f}")
        print(f"Energy gap to ground state: {E_greedy - E_best:.6f}")

# Compare with Simulated Annealing
if "x_sa" in globals() and "E_sa" in globals():
    print("\n=== Greedy vs Simulated Annealing ===")
    print(f"SA energy: {E_sa:.6f}")
    print(f"Greedy energy: {E_greedy:.6f}")
    print(f"Energy difference (Greedy - SA): {E_greedy - E_sa:.6f}")
    print(f"SA time: {elapsed_sa:.4f} seconds")
    print(f"Greedy time: {elapsed_greedy:.4f} seconds")
    print(f"Speedup: {elapsed_sa / elapsed_greedy:.2f}x")
    
    # Check if solutions are the same
    if np.array_equal(x_greedy, x_sa):
        print("Solutions are identical!")
    else:
        print(f"Solutions differ in {np.sum(x_greedy != x_sa)} bits")

=== Greedy Algorithm result ===
Bits (x*): [1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1]
Number of bets in portfolio: 10
Greedy energy: -58.709100
Time to find local optimum: 0.0050 seconds

=== Greedy vs Brute Force ===
Rank of Greedy state among all 2^N configs: 162 (1 = ground state)
Ground state energy (brute force): -62.018248
Energy gap to ground state: 3.309148

=== Greedy vs Simulated Annealing ===
SA energy: -61.492248
Greedy energy: -58.709100
Energy difference (Greedy - SA): 2.783148
SA time: 9.4245 seconds
Greedy time: 0.0050 seconds
Speedup: 1872.24x
Solutions differ in 4 bits


## Summary and typical usage

In this notebook we have:

- Built a realistic **sportsbook portfolio optimisation** instance for a single football match.  
- Encoded the problem as a **QUBO** where:
  - linear terms reward high expected value bets,  
  - quadratic terms penalise correlated / overlapping exposures.  
- Exhaustively searched all \(2^N\) portfolios for a modest-sized universe (N ≈ 20) to obtain the **exact ground state**.
- Mapped the QUBO to an **Ising model** and used **ORBIT** to approximately minimise the Ising energy.
- Compared the ORBIT solution to the brute-force, simulated annealing and greedy optimums and inspected the selected portfolio of bets.
- At scale, ORBIT simulator solution is the only one that can complete. Other algorithms struggle when dealing with 50+ markets.
- Estimation of the ORBIT solution at scale is hard to make as the brute-force algorithm cannot be completed.
- However, its ability to handle larger workloards than other methods makes it the strongest candidate algorithm for sports betting portfolio optimisation.

For a typical user with more compute, the same pipeline can be applied to **larger sets of markets**:

- The data feed would come from live sportsbook APIs rather than csv files of the previously saved API data.  
- The correlation structure \(cov\) can incorporate more sophisticated measures of dependence between markets.  
- ORBIT can be used to explore the energy landscape and propose **high-quality portfolios** within the user’s risk and bankroll constraints.

This notebook is therefore intended as an **end-to-end, validated proof of concept** that a real combinatorial portfolio problem in sports betting can be:
